0. 외부함수 사용을 위해 필요한 라이브러리 불러오기

In [ ]:
import finterstellar as fs
import pandas as pd

In [ ]:
K10_dict = {
    '005930' : '삼성전자',
    '000660' : 'SK하이닉스',
    '051910' : 'LG화학',
    '068270' : '셀트리온',
    '005380' : '현대차',
    '207940' : '삼성바이오로직스',
    '005490' : 'POSCO',
    '028260' : '삼성물산',
    '015760' : '한국전력',
    '035420' : 'NAVER'
}

1. 시가총액이 있는 엑셀 파일 읽기: read_excel()

In [ ]:
path = './data/'     # 파일위치
n = 'K10_info.xlsx'     # 파일명

In [ ]:
file_name = path + n    # 읽어올 파일위치+파일명 지정
stock_info = pd.read_excel(file_name, converters={'code':str})
'''
    pd.read_excel(파일명) - 엑셀파일을 읽어 데이터프레임으로 변환시켜 돌려주는 함수
    converters={'code':str} - 데이터 형식을 '문자'형식으로 지정, 
    생략 시 pd가 이 컬럼을 자동으로 숫자로 인식해 종목코드 앞쪽 0을 삭제해버리게됨
'''
stock_info.set_index('code', inplace=True)
stock_info.head(10)

4. 10개 종목의 시세 데이터 로딩 : create_portfolio_df()

In [ ]:
n = 'K10_prices.xlsx'     # 파일명

In [ ]:
file_name = path + n    # 읽어올 파일위치+파일명 지정
price = pd.read_excel(file_name)
'''
    pd.read_excel(파일명) - 엑셀파일을 읽어 데이터프레임으로 변환시켜 돌려주는 함수
    converters={'code':str} - 데이터 형식을 '문자'형식으로 지정, 
    생략 시 pd가 이 컬럼을 자동으로 숫자로 인식해 종목코드 앞쪽 0을 삭제해버리게됨
'''
price.set_index('trade_date', inplace=True)
price.tail()

In [ ]:
ld = fs.LoadData()

In [ ]:
price = ld.date_formatting(price)
price.tail()

In [ ]:
base_date = '2018-01-02'

In [ ]:
price = price[base_date:]
# 1월1일에 들어있는 NaN 제거
price.head()

In [ ]:
stock_info['outstanding']

In [ ]:
market_cap = pd.DataFrame()
market_cap = price * stock_info['outstanding']
market_cap.head()

In [ ]:
market_cap['K10'] = market_cap.sum(axis=1)
market_cap.tail()

In [ ]:
index = pd.DataFrame()
# 첫 날짜를 기준으로 지수화
index['K10'] = round( market_cap['K10'] / market_cap.loc[base_date, 'K10'], 4 ) * 100
# 지수 = 일자별 시가총액 / 기준일 시가총액 * 100 
index.head()

K10 지수 그래프 그리기

In [ ]:
v = fs.Visualize()

In [ ]:
v.price_view(index, base_date, ['K10'])
# price_view(데이터프레임, 기준일자, [종목코드], (사이즈)) - 챠트 그리기

KOSPI200과 비교

In [ ]:
K200 = ld.make_historical_price_df(path, 'KOSPI 200')

In [ ]:
K200.head()

In [ ]:
index['K200'] = K200.loc[base_date:]
# 1월2일 이후 지수만 idx의 K200 컬럼에 담음
index.head()

In [ ]:
index = index.fillna(method='ffill')
# NaN 값을 앞의 값으로 채움

In [ ]:
v.index_view(index, base_date, ['K200', 'K10'])
# index_view(데이터프레임, 기준일자, [종목코드], (사이즈)) - 비교를 위해 인덱스화한 그래프 그리기